# RRSG 2019 Challenge 

*** 

## Convention

<div class="alert alert-info">
  <strong><span class="badge">BLUE BOX</span></strong> <b> Steps for RRSG 2019 challenge tasks</b> 
</div>

* Running cells followed by the blue boxes is enough to create RRSG 2019 challenge tasks.

<div class="alert alert-warning">
  <strong><span class="badge">YELLOW BOX</span></strong> <b> Supplementary operations </b> 
</div>

* Cells following yellow boxes contain supplementary implementations to generate interactive figures and to perform reconstruction using BART modules. 

<div class="alert alert-danger">
  <strong><span class="badge">RED BOX</span></strong> <b> Warnings to the user</b> 
</div>

Warnings to the user regarding the execution of the cells will appear in the red boxes: 

* The order in which the cells in a Jupyter Notebook are executed matters to ensure that all needed variables are in scope to run upcoming tasks. 
* Running some of the cells more than once may hamper the functionality of the remaining cells. 

## Clarifications

1. How?

2. This notebooks uses [`Script of Scripts (SoS)`](https://vatlab.github.io/sos-docs/) to exchange data between `Octave` and `Python` subkernels. This is achieved by the `%get` magic command  placed at the very beginning of the `Python3` cells. For example: 

```python
%get var_in_octave --from Octave
```

Following this `%get` magic command, `var_in_octave` variable will be also available to `Python3` cells. 

***

## References 

Details regarding the challenge can be found [here](https://ismrm.github.io/rrsg/challenge_one/).

Some of the implementations are adapted to [Ludger Starke's](https://github.com/ISMRM/rrsg/blob/master/challenges/challenge_01/hosted_submissions/reproducibleResearch19_LudgerStarke.zip) submission (MATLAB & BART) for compliance.

Data included in this Notebook is available on [this OSF repository](https://osf.io/xne9w/).

## Initialize neccesary modules

<div class="alert alert-info">
  <strong><span class="badge">INITIALIZE</span></strong> <b> This notebook combines Octave and Python kernels for processing and visualization purposes, respectively. The following 4 cells are responsible for: </b> 
</div>

* Loading Octave's image and optimization packages (included in the container, [please see apt.txt](https://github.com/agahkarakuzu/rrsg2019.om/blob/master/apt.txt)) 
* Importing Python packages (included in the container, [please see postBuild](https://github.com/agahkarakuzu/rrsg2019.om/blob/master/apt.txt))
* Defining two python functions to get a heatmap trace
* Mex two `.c` implementations for calculating density compensation to use them in Octave


<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> The following 4 cells are run on opening to load neccesary modules!</b> 
</div>


In [72]:
%use Octave
disp('Executed automatically');
disp('Cell to load octave packages. To show, select the cell and click arrow up icon in the toolbar.');
pkg load image 
pkg load optim
curdir = pwd;
addpath([curdir filesep 'helpers']);

Executed automatically
Cell to load octave packages. To show, select the cell and click arrow up icon in the toolbar.


In [68]:
%use Python3
# Load python packages
print('Executed automatically');
print('Cell to import python modules. To show, select the cell and click arrow up icon in the toolbar.');
import plotly.graph_objs as go
import plotly_express as px
from plotly import tools, subplots
import numpy as np
import ipywidgets as widgets
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import interact, interactive, fixed, interact_manual
init_notebook_mode(connected=True)
from IPython.display import display, clear_output
from IPython.display import IFrame
from IPython.core.magics.display import Javascript

Executed automatically
Cell to import python modules. To show, select the cell and click arrow up icon in the toolbar.


In [69]:
%use Python3
# This is a Python3 cell to create an interactive figure. 
# Here we use %get magic function to migrate variables from the Octave workspace into Python
print('Executed automatically');
print('Cell to define some Plotly functions. To show, select the cell and click arrow up icon in the toolbar.');

def heatmap_trace(inp, name, xlen, ylen, clrmax, clrmin, clrscale):
    trace = go.Heatmap( z =inp,
                        y = list(range(xlen-1)),
                        x = list(range(ylen-1)),
                        colorscale=clrscale,
                        showscale = True,
                        zmax=clrmax,
                        zmin=clrmin,
                        colorbar=dict(
                        tickfont=dict(
                            size=14,
                            color='white'
                        )),
                        name = name);
    return trace

def heatmap_trace2(inp, name, xlen, ylen, clrscale):
    trace = go.Heatmap( z =inp,
                        y = list(range(xlen-1)),
                        x = list(range(ylen-1)),
                        colorscale=clrscale,
                        showscale = False,
                        name = name);
    return trace

Executed automatically
Cell to define some Plotly functions. To show, select the cell and click arrow up icon in the toolbar.


In [70]:
%use Octave
% Mex c files for gridding by Brian Hargreaves and Philip Beatty 
% http://mrsrl.stanford.edu/~brian/gridding/
disp('Executed automatically');
disp('Cell to MEX c functions for dcf. To show, select the cell and click arrow up icon in the toolbar.');

mex('./helpers/gridlut_mex.c')
mex('./helpers/calcdcflut_mex.c')

Executed automatically
Cell to MEX c functions for dcf. To show, select the cell and click arrow up icon in the toolbar.


## 1. Load ISMRM RRSG 2019 challenge data.

<div class="alert alert-info">
  <strong><span class="badge">1.1</span></strong> <b>Please see this <a href="https://osf.io/xne9w/">OSF page</a> for details.</b> 
</div>

Data is downloaded from the OSF and added to the portable software environment using [`postBuild`](postBuild) configuration file.


In [ ]:
%use Octave
load('/tmp/rrsg_challenge/brain_radial_96proj_12ch.mat');
whos % Show variables in the current scope 

<div class="alert alert-info">
  <strong><span class="badge">1.2</span></strong> <b>Change data order to follow BART's dimension convention</b> 
</div>

<div class="alert alert-danger">
  <strong><span class="badge">Warning</span></strong> <b>The following cell will run automatically after loading data (previous cell).</b><br> If you run it before running the cell above manually, data will be permutted once again and won't be following BART's convention anymore.

--> You can refer to [this documentation](https://buildmedia.readthedocs.org/media/pdf/bart-doc/latest/bart-doc.pdf) for BART's dimension conventions.

</div>


In [ ]:
rawdata = permute(rawdata,[4,3,2,1]); 
trajectory = permute(trajectory,[3,2,1]);
[~,nFE,nSpokes,nCh] = size(rawdata);
imsize_brain = 300;
whos

<div class="alert alert-warning">
  <strong><span class="badge">1.3</span></strong> <b>Display raw data from each channel using Pyhton.</b> 
</div>

In [23]:
%get rawdata --from Octave
%get nFE --from Octave 
%get nSpokes --from Octave
%get nCh --from Octave
print('This (hidden) cell passes variables from Octave to Python')
# This is a data exchange cell Octave --> Python3 

This (hidden) cell passes variables from Octave to Python


In [22]:
# This cell plots rawdata from each channel
# The execution of this cell takes a bit longer than conventional 2D plots (e.g. matplotlib) as each datapoint is represented on a heatmap for interactivity using Plotly. 

rawdata = np.squeeze(rawdata)

fig = subplots.make_subplots(rows=2, cols=6, print_grid=False, horizontal_spacing = 0.02, vertical_spacing = 0.02)

traces = []
iter = 0
for ii in range(2):
    for zz in range(6):
        cur_trace = heatmap_trace(np.log(1+abs(rawdata[:,:,iter])), 'Channel: '+ str(iter+1), nSpokes, nFE, 0.0001, -0, 'Viridis')
        fig.append_trace(cur_trace, ii+1, zz+1)
        iter += 1

fig['layout'].update(height=600, width=800, title=dict(text='<b>Raw data from each channel</b> <br> <i>Hover to see channel number and data points.</i>',font=dict(color="#ffc107")),paper_bgcolor='#000000')

for ii in range(12):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showticklabels = False, ticks = \'\')')

# plot(fig,filename='rawdata.html')
print('This (hidden) cell contains the code regenerating the figure.')
iplot(fig)

This (hidden) cell contains the code regenerating the figure.


In [ ]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Hide', 'ShowSavedFigure', 'RegenerateFigure'],
    description='SELECT:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hides image to use less memory.', 'Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Hide':
        clear_output(wait=True)
    elif val == 'ShowSavedFigure':
        clear_output(wait=True)
        display(IFrame(src='./rawdata.html', width=820, height=600,scrolling='no'))     
    elif val == 'RegenerateFigure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)

        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))

<div class="alert alert-warning">
  <strong><span class="badge">1.4</span></strong> <b>Display k-space data on trajectories</b> 
</div>

In [21]:
# First we will parse data on Octave 

# Use bart to obtain root sum of squares of the rawdata over channels

rd = real(rawdata) + 1i*imag(rawdata);
rd = bart('rss 8', rd);
clr = squeeze(log(rd)); clear rd;
trajx = squeeze(trajectory(1,:,:));
trajy = squeeze(trajectory(2,:,:));
disp('This (hidden) cell parses data on Octave and use BART to visualize it in the next cell.');

This (hidden) cell parses data on Octave and use BART to visualize it in the next cell.


In [24]:
%get trajx --from Octave
%get trajy --from Octave
%get clr --from Octave

# The reason trajx, trajy and trajz are not vectorized in the Octave cell above is that, 
# currently SoS data exchange gets stuck while transferring large vectors. 

trajx = np.reshape(np.array(trajx),(nFE*nSpokes))
trajy = np.reshape(np.array(trajy),(nFE*nSpokes))
clr = np.reshape(np.array(clr),(nFE*nSpokes))

trac = go.Scatter(
    x  = trajx,
    y =  trajy,
    mode='markers',
    marker=dict(
        color = clr, #set color equal to a variable
        colorscale= 'Viridis',
        showscale=True,
        colorbar=dict(
        tickfont=dict(
            size=14,
            color='white'
        ))
    ),
    
)

layout = go.Layout(
    autosize=False,
    width=800,
    height=800,
    paper_bgcolor='#000000',
    plot_bgcolor='#000000',
    xaxis = go.layout.XAxis(
    gridcolor= '#283442',
         tickfont=dict(
            size=14,
            color='white'
        )
    ),
    yaxis = go.layout.YAxis(
    gridcolor= '#283442',
        tickfont=dict(
            size=14,
            color='white'
        )
    ),
    hovermode = 'closest',
    title=dict(text='<b>Raw data projected on k-space trajectory</b> <br> <i>Hover to see the locations of each sample</i>',font=dict(color="#ffc107"))
   
)

fig = go.Figure(data=[trac], layout=layout)

#plot(fig, filename='kspace.html')
print('This (hidden) cell contains the code regenerating the figure.')
iplot(fig)

This (hidden) cell contains the code regenerating the figure.


In [ ]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Hide', 'ShowSavedFigure', 'RegenerateFigure'],
    description='SELECT:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hides image to use less memory.', 'Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Hide':
        clear_output(wait=True)
    elif val == 'ShowSavedFigure':
        clear_output(wait=True)
        display(IFrame(src='./kspace.html', width=820, height=600,scrolling='no'))     
    elif val == 'RegenerateFigure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)

        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))

## 2. Estimate coil sensitivities using BART
***

<div class="alert alert-info">
  <strong><span class="badge">2.1</span></strong> <b>Use BART to estimate coil sensitivities</b> 
  <br><br>To re-generate figures, you need to select CALDIR.  
</div>

In [64]:
%use Python3

Stgl = widgets.ToggleButtons(
    options=['Idle', 'ECALIB', 'CALDIR'],
    description='SELECT:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Make a selection to have sensitivity maps in scope.', 'Use ECALIB', 'Use CALDIR'],
)

@interact
def Stoggle(val=Stgl):
    if val == 'Idle':
        print('Please make a selection to calculate S. Cells will be run automatically upon your selection.')
    elif val == 'ECALIB':
        print('Calculate sensitivity profiles using BART\'s ECALIB opiton.')
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.get_selected_index()+2)"))      
    elif val == 'CALDIR':
        print('Calculate sensitivity profiles using BART\'s CALDIR opiton.')
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.get_selected_index()+2)"))

interactive(children=(ToggleButtons(button_style='info', description='SELECT:', options=('Idle', 'ECALIB', 'CA…

In [33]:
%use Python3 
print('This (hidden) cell transfers widget state from Python to Octave.')
tmp = Stoggle.widget.children[0]
S_select =tmp.index

This (hidden) cell transfers widget state from Python to Octave.


In [ ]:
%use Octave 
%get S_select --from Python3

% This cell will be run automatically based upon your selection from the blue toggle above. 

% Perform NUFFT to interpolate data onto cartesian grid. 

im = bart('nufft -d300:300:1 -a -t',trajectory,rawdata);

% Transform sensitivity maps to cartesian k-space using FFT
% -u denotes unitary transform
% 7 sets the bitmask level

im_ks = bart('fft -u 7', im);

% For details regarding BART's ECALIB, please see the implementation notes section.

if S_select == 1
    disp('Calculating S using ECALIB.');
    calib = bart('ecalib -r 20 -m1 -I',im_ks);
    calib_disp = squeeze(abs(calib));
    disp('--> Done...')
elseif S_select == 2
    disp('Calculating S using CALDIR.');
    calib = bart('caldir 20',im_ks);
    disp('--> Done...'); 
    calib_disp = squeeze(abs(calib));
end

<div class="alert alert-warning">
  <strong><span class="badge">2.2</span></strong> <b>Display sensitivity profiles per channel.</b>
</div>


<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> Depends on 2.1</b> 
</div>

In [28]:
%use Python3
%get calib_disp --from Octave
print('This (hidden) cell passes variables from Octave to Python')
# This is a data exchange cell. Octave --> Python3

This (hidden) cell passes variables from Octave to Python


In [30]:
calib = calib_disp
fig = subplots.make_subplots(rows=3, cols=4, print_grid=False, horizontal_spacing = 0.02, vertical_spacing = 0.02)

traces = []
iter = 0
for ii in range(3):
    for zz in range(4):
        mx = np.max(np.log(1+abs(calib[:,:,iter])))
        mn = np.min(np.log(1+abs(calib[:,:,iter])))
        cur_trace = heatmap_trace(np.log(1+abs(calib[:,:,iter])), 'Channel: '+ str(iter+1), 300, 300, mn,mx , 'Viridis')
        fig.append_trace(cur_trace, ii+1, zz+1)
        iter += 1

fig['layout'].update(height=750, width=800, title=dict(text='<b>Sensitivity profile of each channel</b> <br> <i>Hover to see channel number and data points.</i>',font=dict(color="#ffc107")),paper_bgcolor='#000000')

for ii in range(12):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showticklabels=False,showline = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

#plot(fig,filename='calibrations.html')
print('This (hidden) cell contains the code regenerating the figure.');
iplot(fig)

This (hidden) cell contains the code regenerating the figure.


In [65]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Hide', 'ShowSavedFigure', 'RegenerateFigure'],
    description='SELECT:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hides image to use less memory.', 'Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Hide':
        clear_output(wait=True)
    elif val == 'ShowSavedFigure':
        clear_output(wait=True)
        display(IFrame(src='./calibrations.html', width=820, height=600,scrolling='no'))     
    elif val == 'RegenerateFigure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))

interactive(children=(ToggleButtons(button_style='warning', description='SELECT:', options=('Hide', 'ShowSaved…

## 3. Reconstruction function 

<div class="alert alert-warning">
  <strong><span class="badge">3.1</span></strong> <b>Use SENSE recon (both cartesian and non-cartesian) provided by BART for demonstration</b> 
</div>

<div class="alert alert-warning">
  <strong><span class="badge">3.1.1</span></strong> <b>Cartesian SENSE (BART) using k-space data obtained by fft of nufft</b> 
</div>

In [ ]:
% Use gridded data for SENSE recon --> BART PICS 
bart_SENSE = bart('pics -l2', im_ks, calib);

<div class="alert alert-warning">
  <strong><span class="badge">3.1.2</span></strong> <b>Non-Cartesian SENSE (BART) using provided data</b> 
</div>

In [ ]:
% Use non-cartesian SENSE recon --> BART PICS 
bart_SENSE2 = bart('pics -t',trajectory, rawdata, calib);

<div class="alert alert-warning">
  <strong><span class="badge">3.1.3</span></strong> <b>Compare BART reconstructions from 3.0.1 and 3.0.2</b> 
</div>

<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> Depends on 3.1.1 and 3.1.2</b> 
</div>


In [25]:
%use Python3
%get bart_SENSE --from Octave 
%get bart_SENSE2 --from Octave

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False,vertical_spacing = 0.02)

fig.append_trace(heatmap_trace2(abs(bart_SENSE), 'BART pics -l2', 300, 300,'Greys'),1,1)
fig.append_trace(heatmap_trace2(abs(bart_SENSE2), 'BART pics -t', 300, 300,'Greys'),1,2)

fig['layout'].update(height=400, width=600, title=dict(text='<b>Cartesian vs non-cartesian SENSE in BART</b>',font=dict(color="#ffc107")),paper_bgcolor='#000000')

for ii in range(2):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showticklabels = False, ticks = \'\')')

#plot(fig,filename='bart_cart_non.html')
print('This (hidden) cell contains the code regenerating the figure.')
iplot(fig)

This (hidden) cell contains the code regenerating the figure.


In [66]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Hide', 'ShowSavedFigure', 'RegenerateFigure'],
    description='SELECT:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hides image to use less memory.', 'Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Hide':
        clear_output(wait=True)
    elif val == 'ShowSavedFigure':
        clear_output(wait=True)
        display(IFrame(src='./bart_cart_non.html', width=820, height=600,scrolling='no'))     
    elif val == 'RegenerateFigure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)
        display(Javascript("Jupyter.notebook.execute_cell(IPython.notebook.get_selected_index()+1)"))

interactive(children=(ToggleButtons(button_style='warning', description='SELECT:', options=('Hide', 'ShowSaved…

<div class="alert alert-info">
  <strong><span class="badge">3.2</span></strong> <b>Subsample provided data by factors of 2, 3 and 4.</b>
   <br><br>The cell below will be run automatically on opening.
</div>


In [ ]:
# ===========================================
# Subsample data 

# Please see HelperFunctions folder for subSample.m 
# The cell below will be run automatically on opening.
# You don't have to re-run unless changes to the variables made on purpose. 
# ===========================================

[outRD_x2, outTR_x2] = subSample(rawdata,trajectory,2,nSpokes);

[outRD_x3, outTR_x3] = subSample(rawdata,trajectory,3,nSpokes);

[outRD_x4, outTR_x4] = subSample(rawdata,trajectory,4,nSpokes);

<div class="alert alert-warning">
  <strong><span class="badge">3.3</span></strong> <b>Use non-cartesian SENSE recon (BART) to observe the effects of subsampling</b>
</div>


<div class="alert alert-warning">
  <strong><span class="badge">3.3.1</span></strong> <b>Perform non-cartesian SENSE recon (BART)</b>
</div>


In [ ]:
% BART non-cartesian sense outputs 
bart_SENSE2_x2 = bart('pics -t',outTR_x2, outRD_x2, calib);
bart_SENSE2_x3 = bart('pics -t',outTR_x3, outRD_x3, calib);
bart_SENSE2_x4 = bart('pics -t',outTR_x4, outRD_x4, calib);

<div class="alert alert-warning">
  <strong><span class="badge">3.3.2</span></strong> <b>Display: Compare reconstructed images</b>
</div>


<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> Depends on 3.3.1</b> 
</div>


In [27]:
%use Python3
%get bart_SENSE --from Octave 
%get bart_SENSE2_x2 --from Octave
%get bart_SENSE2_x3 --from Octave
%get bart_SENSE2_x4 --from Octave
print('This (hidden) cell passes variables from Octave to Python')
# Data exchange cell. Octave --> Python3

This (hidden) cell passes variables from Octave to Python


In [26]:
%use Python3
fig = subplots.make_subplots(rows=2, cols=2, print_grid=False,vertical_spacing = 0.02, horizontal_spacing=0.04)

fig.append_trace(heatmap_trace2(abs(bart_SENSE2), 'Original', 300, 300,'Greys'),1,1)
fig.append_trace(heatmap_trace2(abs(bart_SENSE2_x2), '2X', 300, 300,'Greys'),1,2)
fig.append_trace(heatmap_trace2(abs(bart_SENSE2_x3), '3X', 300, 300,'Greys'),2,1)
fig.append_trace(heatmap_trace2(abs(bart_SENSE2_x4), '4X', 300, 300,'Greys'),2,2)

fig['layout'].update(height=600, width=600, title=dict(text='<b>Comparison of BART non-cartesian SENSE outputs <br> at different undersampling rates</b>',font=dict(color="#ffc107")),paper_bgcolor='#000000')

for ii in range(4):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, showline=False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

#plot(fig,filename='sub_bart_compare.html')
print('This (hidden) cell contains the code regenerating the figure.');
iplot(fig)

This (hidden) cell contains the code regenerating the figure.


In [ ]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Hide', 'ShowSavedFigure', 'RegenerateFigure'],
    description='SELECT:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hides image to use less memory.', 'Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Hide':
        clear_output(wait=True)
    elif val == 'ShowSavedFigure':
        clear_output(wait=True)
        display(IFrame(src='./sub_bart_compare.html', width=820, height=600,scrolling='no'))     
    elif val == 'RegenerateFigure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))


<div class="alert alert-info">
  <strong><span class="badge">Section 4</span></strong> <b>Implementation of the reconstruction algorithm</b>
</div>

* To better relate implementation to the paper, reminders and implementation notes (following the original notation) are added.


<p style="color:navy;font-size:16px; background-color:powderblue; border-left: 6px solid navy; display: inline-block"> &nbsp; <i class="fa fa-info-circle fa-1x" style="color:darkblue"></i>  <b>Reminder: Figure-1 from the paper&nbsp;</b></p> <br>

<br>

<div class="row">
<div class="col-sm-5" style="padding-right:5px">
<img src="https://wol-prod-cdn.literatumonline.com/cms/attachment/2594c4ea-88e5-4078-b0ea-9893a6554853/mfig001.jpg" style="width:500px;height:400px" align="left">
</div>
<div class="col-sm-7" style="background-color:powderblue">
<p style="font-size:12px; padding-left:15px"> <b>Implementation of iterative image reconstruction.</b><br><br> Conjugate gradient (CG) iteration is controlled by the central CG process. It is initialized by MR data originating from N receiver channels (1,2,3,…N), acquired with an arbitrary k‐space trajectory. <br> Separately for each channel, these data undergo processing similar to conventional gridding reconstruction, i.e., sampling density correction (D), and resampling along a Cartesian grid, followed by FFT (FT1). The resulting images are individually multiplied by complex conjugate coil sensitivity and summed. After subsequent intensity correction (I), the sum image represents the vector a as defined by Eq. [25]. After initialization with a, the CG process iteratively calculates a progression of images, which converges towards exact reconstruction. For each iteration step, a current residuum image vector needs to be multiplied by the matrix IEH DEI. This is performed by the loop which starts from the CG box. After initial intensity correction (I), the processing is continued separately for each receiver coil. First, the intensity corrected residuum image is multiplied by individual coil sensitivity (Si). The results are transformed into k‐space by FFT and resampled along the experimental k‐space trajectory (FT2), resulting in a set of multiple‐coil k‐space data similar to that obtained experimentally. The following steps are equivalent to those carried out with the original data, yielding an intermediate image, which is fed back into the CG process. Here a refined approximate solution is calculated, which serves for further refinement by continued iteration. <br>As soon as the current approximation is sufficiently accurate, it is output and undergoes final intensity correction and k‐space filtering.</p>
</div>
</div>


<p style="color:navy;font-size:16px; background-color:powderblue; border-left: 6px solid navy; display:inline-block"> &nbsp; <i class="fa fa-info-circle fa-1x" style="color:darkblue"></i>  <b> Implementation notes &nbsp; </b></p>

This section relates the present implementation with the Figure-1 using the same notations. 

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;FT1&nbsp; </p><b> operation is implemented using BART's NUFFT:</b> <br>

This function transforms non-cartesian k-space data (`rawdata` and `trajectory`) into image domain (`im`): 
```octave 
tmp = bart('nufft -d300:300:1 -a -t',traj,inp); 

% -d denotes dimensions in x:y:z
% -a sets the transform type to adjoint
% -t enables Toeplitz embedding
%  trajectory is the k-space locations of the acquired samples
%  rawdata is the sampled raw data

```

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;S<sub>$\gamma$</sub>&nbsp; </p> <b> coil sensitivities are estimated using BART's ECALIB. See section 2.1</b>

This function creates sensitivity maps (stored in the `calib` variable) from the raw k-space data on cartesian grid (sens_maps_ks):

<p>&#9658; <b>Option 1</b></p>

```octave
calib = bart('ecalib -m1 -I',sens_maps_ks);

% -I enables intensity correction
% -m1 sets number of maps to one 
```

<p>&#9658; <b>Option 2</b></p>

```octave
calib = bart('caldir 20',sens_maps_ks);

% Calibration of sensitivities directly from the center of the k-space.
% 20 sets the size of fully-sampled calibration region.
```

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;FT2&nbsp; </p> <b>is implemented using BART's NUFFT:</b> <br>

This function transforms image `im` back to a non-uniform k-space of (radial, which is defined by `trajectory` variable in this case) `nu_ks`:

```octave
nu_ks = bart('nufft',traj,im); 
```

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;D&nbsp; </p> <b>is the operation of scaling rawdata with density correction matrix.</b> <br>

To obtain density correction factor `dcf` pertaining to the `rawdata`, two options are provided.


<p>&#9658; <b>Option 1</b></p>

```octave
dcf = abs(trajectory(1,:,:) + 1i*trajectory(2,:,:));
dcf = dcf/max(dcf(:));

% Magnitude of the trajectory data is scaled by the max.
% to be used as a rough DCF approximation. 
```

<p>&#9658; <b>Option 2</b></p>

Submodules (written in C) provided by [Brian Hargreaves's gridding functions](http://mrsrl.stanford.edu/~brian/gridding/) can be selected:

```octave
dcf = calcdcflut(trajectory,300); 

% trajectory is the k-space locations of the non-cartesian rawdata
% 300 is the recon matrix size (300x300)
```


* <p style="border:1px;border-style:solid;display:inline-block; border-radius:50%"> &nbsp;Sum&nbsp; </p> <b>is implemented using BART's RRS:</b> <br>

This function simply performs sum of squares combination: 

```python
sum = bart('rss bitmask',multi_channel_input); 
```

* <p style="border:1px;border-style:solid;display:inline-block;"> &nbsp;F&nbsp; </p> <b>is the k-space filtering, defined in 3.6</b> <br>


<div class="alert alert-info">
  <strong><span class="badge">3.2</span></strong> <b>Calculate density compensation factor for operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;D&nbsp; </p> </b>
    <br><br>To re-generate figures, you need to select (Fast) Simple.
</div>

Two options are made available. 

<p>&#9658; <b>Option 1</b></p>

An simple approximation. 

<p>&#9658; <b>Option 2</b></p>

The submodules (written in C, mexed for Octave) provided by [Brian Hargreaves's gridding functions](http://mrsrl.stanford.edu/~brian/gridding/) to provide a good example of interoperability.  

In [ ]:
%use Python3

tgl = widgets.ToggleButtons(
    options=['Idle', '(Fast) Simple', '(Slow) DCF in C'],
    description='SELECT:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Make a selection to have DCF in scope.', '(Fast) Simple', '(Slow) DCF in C'],
)

@interact
def dcftoggle(val=tgl):
    if val == 'Idle':
        print('Please make a selection to calculate DCF. Cells will be run automatically upon your selection.')
    elif val == '(Fast) Simple':
        print('This will calculate DCF quickly.')
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.get_selected_index()+2)"))      
    elif val == '(Slow) DCF in C':
        print('This operation may take ~2 minutes. Please wait until you see Done.. message in the next cell.')
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.get_selected_index()+2)"))


In [35]:
%use Python3 
print('This (hidden) cell transfers widget state from Python to Octave.')
tmp = dcftoggle.widget.children[0]
dcf_select =tmp.index

This (hidden) cell transfers widget state from Python to Octave.


In [ ]:
%use Octave 
%get dcf_select --from Python3

# ===========================================
# Calculate DCF 

# dcf_select variable is determined by the blue toggle buttons above.
# This cell will be run automatically upon your selection. 
# ===========================================


if dcf_select == 1
   
    disp('Simple DCF estimation')
    dcf = abs(trajectory(1,:,:) + 1i*trajectory(2,:,:));
    dcf = dcf/max(dcf(:));
   disp('--> Done..');
   
elseif dcf_select==2
   
    disp('Calculating DCF using code written in C...')
    dcf = calcdcflut(trajectory,300); 
    dcf = reshape(dcf,[3 nFE nSpokes]);
    disp('--> Done..');
    
end


<div class="alert alert-info">
  <strong><span class="badge">3.3</span></strong> <b>Calculate intensity correction matrix for operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;I&nbsp; </p></b>
</div>


<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> Depends on 2.1</b> 
</div>

In [37]:
%use Octave

# ===========================================
# Calculate I !!!! DO NOT FORGET TO RUN !!!

# Calculate intensity correction matrix.
# ===========================================

# Root sum of square of sensitivities (estimated in section 2.1) from 12 channels 
I = abs(bart('rss 8',calib.*calib));
I = 1./sqrt(I);

In [34]:
%use Python3
%get I --from Octave

# ===========================================
# Generate interactive figure displaying I 
# ===========================================

trace  = heatmap_trace2(abs(I), 'I', 300, 300,'Viridis')
axis_template = dict(showgrid = False, zeroline = False,
             linecolor = 'black', showticklabels = False,
             ticks = '' )
layout = dict(height=500, width=500, title=dict(text='<b>Intensity correction matrix</b>',font=dict(color="skyblue")),paper_bgcolor='#000000',xaxis=axis_template,yaxis=axis_template)
figure = dict(data=[trace],layout=layout)

#plot(figure,filename='incor.html')
iplot(figure)

<div class="alert alert-info">
  <strong><span class="badge">3.4</span></strong> <b>Define operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;E&nbsp; </p></b>
</div>

* Function prototype

```octave
function E = opE(inp,S,traj)

```

* Input arguments 
     * **inp:** The image data (300,300)
     * **S:** Sensitivity profiles from 12 channels (300,300,12)
     * **traj:** k-space coordinates (3,512,96)

* Output 
     * **E** Please see the Figure-1 above
     
**The following cell executes automatically to ensure that function definition exists.**

In [ ]:
function E = opE(inp,S,traj)

    tmp = S.*inp; % Multiply the intensity corrected image with coil sensitivities. This will produce one image per channel stored in tmp variable

    E = bart('nufft',traj,tmp); % Transform back to the non-uniform k-space (see implementation notes for details)

end

<div class="alert alert-info">
  <strong><span class="badge">3.5</span></strong> <b>Define operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;E<sup>H</sup>&nbsp; </p></b>
</div>

* Function prototype

```octave
function EH = opEH(inp,S,traj,imsz)

```

* Input arguments 
     * **inp:** Non-cartesian k-space samples from multiple channels (512,96,12) 
     * **S:** Sensitivity profiles from 12 channels
     * **traj:** k-space coordinates (3,512,96)
     * **imsz:** size of the reconstructed image

* Output 
     * **EH** Please see the Figure-1 above
     
**The following cell executes automatically to ensure that function definition exists.**

In [ ]:
function EH = opEH(inp,S,traj,imsz)
  
  tmp = squeeze((1+1i)*ones(size(S)));

  tmp = bart(['nufft -d' num2str(imsz) ':' num2str(imsz) ':1 -a -t'],traj,inp); % Adjoint NUFFT to image domain (see implementation notes)
  
  Sstar = conj(S(:,:,1,:)); %  Complex conjugate of sensitivity profiles (see Fig. 1)
  
  EH = bart('rss 8 ',tmp.*Sstar); % Images scaled by complex conjugate of sensitivity profiles and SOS combined
  
end

<div class="alert alert-info">
  <strong><span class="badge">3.6</span></strong> <b>Define operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;F&nbsp;</p></b>
    <br><br> 
The following cell executes automatically to ensure that function definition exists.
</div>


In [ ]:
function out = opF(im,beta,r)
    
    ks = 1/sqrt(numel(im))*fftshift(fftn(ifftshift(im)));
    
    dim = size(ks);
    center = (dim + 1)/2;
    [x,y] = meshgrid(-(center(2)-1):(dim(2)-center(2)),-(center(1)-1):(dim(1)-center(1)));
    kRadius = sqrt(x.^2 + y.^2);

    f_k = 0.5 + 1/pi.*atan(beta.*((r-kRadius)/r)); 

    out = sqrt(numel(ks.*f_k))*fftshift(ifftn(ifftshift(ks.*f_k)));

end

<div class="alert alert-info">
  <strong><span class="badge">3.7</span></strong> <b>Define iterative algorithm</b>
        <br><br> 
The following cell executes automatically to ensure that function definition exists.
</div>

In [ ]:
function [b,deltas,refdif] = cg_solve(a,I,S,dcf,imsz,maxstep,trajectory,tol,ref)

    p = a;
    r = a;
    b = zeros(imsz,imsz);

    deltas = zeros(maxstep,1);
    refdif = zeros(maxstep,1);

    for ii=1:maxstep

        disp(['Iteration -->' num2str(ii)]);

        delta = r(:)'*r(:)/(a(:)'*a(:));

        disp(delta);
        deltas(ii) = delta;

        E = opE(p,S,trajectory);

        q = opEH(E,S,trajectory,imsz);

        % dot(r,r) is equivalent to r(:)'*r(:). Used dot for easy reading. 

        term = dot(r,r)/dot(p,q);

        b = b + term*p;

        rprev = r;

        r = r - term*q;

        term2 = dot(r,r)/dot(rprev,rprev);

        p = r + term2*p;

        if (delta <= tol)
            printf ("%d has been reached\n", tol);
            break;
        endif

        if nargout ==3 && nargin ==9 
        % This means that ref image is passed. 

        refN = ref(:)'*ref(:);
        ref_dif = I.*b - ref;
        refdif(ii) = sqrt((ref_dif(:)'*ref_dif(:))./refN);

        end 


    end
    
    
   
end


<div class="alert alert-info">
  <strong><span class="badge">3.8</span></strong> <b>Define main call to the iterative solution</b>
        <br><br> 
The following cell executes automatically to ensure that function definition exists.
</div>

In [ ]:
function [im,deltas,a,refdif] = main_sense(rawdata, calib, trajectory,I,dcf,imsz,maxiter,tol,ref)

    szz = size(rawdata);
    szz = szz(4);
    
    dcf = repmat(dcf,[1,1,1,szz]);
    
    a = I.*opEH(dcf.*rawdata,calib,trajectory,imsz);
    
     if nargout ==4 && nargin ==9 
    % This means that ref image is passed. 
        disp('Reference image is passed.');
        [im,deltas,refdif] = cg_solve(a,I,calib,dcf,imsz,maxiter,trajectory,tol,ref);
    
    else
    
        [im,deltas] = cg_solve(a,I,calib,dcf,imsz,maxiter,trajectory,tol);
    
    end 
    
    im = opF(im.*I,100,60);
    
    
end

<div class="alert alert-info">
  <strong><span class="badge">4.1</span></strong> <b> Generate figures</b>
</div>

**Selected options to generate saved images:** 

* `DCF`: **(FAST) Simple**
* `Sensitivity profiles`: **CALDIR**

<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> Depends on <strong><span class="badge">2.1</span></strong> &  
    <strong><span class="badge">3.2 to 3.8</span></strong></b> 
</div>

<div class="alert alert-info">
  <strong><span class="badge">4.1.1</span></strong> <b>Reference reconstruction</b>
</div>

In [ ]:
# ===========================================
# Generate reference image 

# Set iteration to 30 for reference. 
# ===========================================

% ==> OPTION-1: RE-RUN to create reference output. ===================== (SLOWER)

%{ 

[im_ref,deltas_ref,init_ref] = main_sense(rawdata, calib, trajectory,I,dcf,imsize_brain,30,10^(-12));
save ref_recon.mat im_ref deltas_ref init_ref

%}

% ==> OPTION-2: LOAD pre-calculated output ===================== (FASTER)
% Pre calculated outputs are fetched from (and are available) on OSF https://osf.io/xne9w/. 

load('ref_recon.mat'); 

<div class="alert alert-info">
  <strong><span class="badge">4.1.2</span></strong> <b>Reconstruction of sub-sampled data</b>
</div>


In [53]:
# ===========================================
# Perform reconstruction on subsampled data 

# Set iteration to 15. 
# ===========================================

% ==> OPTION-1: RE-RUN to create outputs for sub-sampled data. ===================== (SLOWER)

%{
% Less iterations w/o subsampling 
[im,deltas,init,refdif] = main_sense(rawdata,calib,trajectory,I,dcf,imsize_brain,15,0.01,im_ref);

% Reconstruction fo sub-sampled data 
[im_x2,deltas_x2,init_x2,refdif_x2] = main_sense(outRD_x2,calib,outTR_x2,I,dcf(:,:,1:2:end),imsize_brain,15,0.01,im_ref);
[im_x3,deltas_x3,init_x3,refdif_x3] = main_sense(outRD_x3,calib,outTR_x3,I,dcf(:,:,1:3:end),imsize_brain,15,0.01,im_ref);
[im_x4,deltas_x4,init_x4,refdif_x4] = main_sense(outRD_x4,calib,outTR_x4,I,dcf(:,:,1:4:end),imsize_brain,15,0.01,im_ref);

save sub_recon.mat im deltas init refdif im_x2 deltas_x2 init_x2 refdif_x2 im_x3 deltas_x3 init_x3 refdif_x3 im_x4 deltas_x4 init_x4 refdif_x4 
%}

% ==> OPTION-2: LOAD pre-calculated outputs ===================== (FASTER)
% Pre calculated outputs are fetched from (and are available) on OSF https://osf.io/xne9w/. 

load('sub_recon.mat');

<div class="alert alert-info">
  <strong><span class="badge">4.1.3</span></strong> <b>Reconstruction of single-coil images</b>
</div>


In [73]:
# ===========================================
# Reconstruct single coil images. 

# Just perform inverse gridding and take the image from the first coil. 
# ===========================================

% ==> OPTION-1: RE-RUN to create outputs for single coil images. ===================== (SLOWER)

%{
single_coil = bart('nufft -i -t', trajectory, rawdata);
single_coil = abs(squeeze(single_coil(:,:,:,1)));

single_coil_x2 = bart('nufft -i -t', outTR_x2, outRD_x2);
single_coil_x2 = abs(squeeze(single_coil_x2(:,:,:,1)));

single_coil_x3 = bart('nufft -i -t', outTR_x3, outRD_x3);
single_coil_x3 = abs(squeeze(single_coil_x3(:,:,:,1)));

single_coil_x4 = bart('nufft -i -t', outTR_x4, outRD_x4);
single_coil_x4 = abs(squeeze(single_coil_x4(:,:,:,1)));

save single_recon.mat single_coil single_coil_x2 single_coil_x3 single_coil_x4
%}

% ==> OPTION-2: LOAD pre-calculated outputs ===================== (FASTER)
% Pre calculated outputs are fetched from (and are available) on OSF https://osf.io/xne9w/. 

load('single_recon.mat');

<div class="alert alert-info">
  <strong><span class="badge">4.1.4</span></strong> <b>Generate Figure-4</b>
    
  If you cannot see Figure-4, Kernel --> Restart & Clear Output
</div>

<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> If you'd like to regenerate the figure, operation depends on <strong><span class="badge">4.1.1</span></strong> &  
    <strong><span class="badge">4.1.2</span></strong></b> 
</div>

In [46]:
%use Python3
%get deltas --from Octave
%get deltas_x2 --from Octave
%get deltas_x3 --from Octave
%get deltas_x4 --from Octave
%get refdif --from Octave
%get refdif_x2 --from Octave
%get refdif_x3 --from Octave
%get refdif_x4 --from Octave

print('This (hidden) cell transfers data from Octave to Python.')

# Transfer outputs from Octave to Python for visualizaiton. 

This (hidden) cell transfers data from Octave to Python.


In [47]:
iterations = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15];

sub2_1 = go.Scatter(
    x = iterations,
    y = np.log10(deltas),
    name = 'R=1',
    text = 'R=1',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(0,114,189)'),
    legendgroup="group"
)

sub2_2 = go.Scatter(
    x = iterations,
    y = np.log10(deltas_x2),
    name = 'R=2',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(217,83,25)'),
    legendgroup="group"
)

sub2_3 = go.Scatter(
    x = iterations,
    y = np.log10(deltas_x3),
    name = 'R=3',
    text = 'R=3',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(237,177,32)'),
    legendgroup="group"
)

sub2_4 = go.Scatter(
    x = iterations,
    y = np.log10(deltas_x4),
    name = 'R=4',
    text = 'R=4',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(126,47,142)'),
    legendgroup="group"
)

# -----

sub1_0 = go.Scatter(
    x = iterations,
    y = np.log10(refdif),
    name = 'R=1',
    text = 'R=1',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(0,114,189)',dash='dot'),
    legendgroup="group2"
)

sub1_1 = go.Scatter(
    x = iterations,
    y = np.log10(refdif_x2),
    name = 'R=2',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(217,83,25)',dash='dot'),
    legendgroup="group2"
)

sub1_2 = go.Scatter(
    x = iterations,
    y = np.log10(refdif_x3),
    name = 'R=3',
    text = 'R=3',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(237,177,32)',dash='dot'),
    legendgroup="group2"
)

sub1_3 = go.Scatter(
    x = iterations,
    y = np.log10(refdif_x4),
    name = 'R=4',
    text = 'R=4',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(126,47,142)',dash='dot'),
    legendgroup="group2"
)

fig = tools.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes = True)

fig.append_trace(sub2_1, 1, 1)
fig.append_trace(sub2_2, 1, 1)
fig.append_trace(sub2_3, 1, 1)
fig.append_trace(sub2_4, 1, 1)

fig.append_trace(sub1_0, 2, 1)
fig.append_trace(sub1_1, 2, 1)
fig.append_trace(sub1_2, 2, 1)
fig.append_trace(sub1_3, 2, 1)

axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white") )

fig.update_layout(height=800, width=600, title=dict(text='<b>Repeat <a href="https://wol-prod-cdn.literatumonline.com/cms/attachment/0ef75948-26f4-445b-9014-e5e034c9c7a1/mfig004.jpg" target="_blank">Figure-4</a> of the original paper.</b>',font=dict(color="skyblue")),paper_bgcolor='#000000', xaxis=axis_template,yaxis=axis_template,xaxis2 =axis_template ,yaxis2=axis_template)
fig.update_xaxes(title=dict(text='Iterations',font=dict(color="white")), row=2, col=1)
fig.update_yaxes(title=dict(text='Log<sub>10</sub>∆',font=dict(color="white")), row=1, col=1)
fig.update_yaxes(title=dict(text='Log<sub>10</sub>' + u"\u03B4",font=dict(color="white")), row=2, col=1)
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))


iplot(fig)
#plot(fig,filename='fig4.html')
print('This (hidden) cell (re)generates Figure-4')

This (hidden) cell (re)generates Figure-4


In [73]:
%use Python3
import time, sys
tb = widgets.ToggleButtons(
    options=['Show Saved Figure', 'Regenerate Figure'],
    description='FIG4:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Show Saved Figure':
        clear_output(wait=True)
        display(IFrame(src='./fig4.html', width=600, height=800,scrolling='no'))     
    elif val == 'Regenerate Figure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))


interactive(children=(ToggleButtons(button_style='info', description='FIG4:', options=('Show Saved Figure', 'R…

<div class="alert alert-info">
  <strong><span class="badge">4.1.5</span></strong> <b>Generate Figure-5</b>
  
   If you cannot see Figure-5, Kernel --> Restart & Clear Output
</div>

<div class="alert alert-danger">
  <strong><span class="badge">RUN FIRST</span></strong> <b> If you'd like to regenerate the figure, operation depends on <strong><span class="badge">4.1.3</span></strong> to  
    <strong><span class="badge">4.1.2</span></strong></b> 
</div>

In [87]:
%use Python3

%get im --from Octave
%get im_x2 --from Octave
%get im_x3 --from Octave
%get im_x4 --from Octave
%get single_coil --from Octave
%get single_coil_x2 --from Octave
%get single_coil_x3 --from Octave
%get single_coil_x4 --from Octave
%get init --from Octave
%get init_x2 --from Octave
%get init_x3 --from Octave
%get init_x4 --from Octave

print('This (hidden) cell transfers data from Octave to Python.');

This (hidden) cell transfers data from Octave to Python.


In [88]:
%use Python3 

def normabs(inp):
    tmp  = abs(inp)
    nor = (tmp-np.amin(tmp))/(np.amax(tmp)-np.amin(tmp))
    return nor

fig = subplots.make_subplots(rows=4, cols=3, print_grid=False,vertical_spacing = 0.02, horizontal_spacing=0.04)

fig.append_trace(heatmap_trace2(normabs(single_coil), 'R1-SingleCoil', 300, 300,'Viridis'),1,1)
fig.append_trace(heatmap_trace2(normabs(init), 'R1-Initial', 300, 300,'Viridis'),1,2)
fig.append_trace(heatmap_trace2(normabs(im), 'R1-Final', 300, 300,'Viridis'),1,3)

fig.append_trace(heatmap_trace2(normabs(single_coil_x2),'R2-SingleCoil', 300, 300,'Viridis'),2,1)
fig.append_trace(heatmap_trace2(normabs(init_x2), 'R2-Initial', 300, 300,'Viridis'),2,2)
fig.append_trace(heatmap_trace2(normabs(im_x2), 'R2', 300, 300,'Viridis'),2,3)

fig.append_trace(heatmap_trace2(normabs(single_coil_x3),'R3-SingleCoil', 300, 300,'Viridis'),3,1)
fig.append_trace(heatmap_trace2(normabs(init_x3), 'R3-Initial', 300, 300,'Viridis'),3,2)
fig.append_trace(heatmap_trace2(normabs(im_x3), 'R3-Final', 300, 300,'Viridis'),3,3)

fig.append_trace(heatmap_trace2(normabs(single_coil_x4),'R4-SingleCoil', 300, 300,'Viridis'),4,1)
fig.append_trace(heatmap_trace2(normabs(init_x4), 'R4-Initial', 300, 300,'Viridis'),4,2)
fig.append_trace(heatmap_trace2(normabs(im_x4), 'R4-Final', 300, 300,'Viridis'),4,3)

fig.update_layout(height=800, width=800, title=dict(text='<b>Repeat <a href="https://wol-prod-cdn.literatumonline.com/cms/attachment/3b56d3f8-c3f4-4ba0-afe5-1d77ecb1c5a3/mfig005.jpg" target="_blank">Figure-5</a> of the original paper.</b> <br> &nbsp;',font=dict(color="skyblue")),paper_bgcolor='#000000')

for ii in range(12):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, showline=False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

fig.update_xaxes(title=dict(text='<b>Single Coil</b>',font=dict(color="white")), row=1, col=1, side='top')
fig.update_xaxes(title=dict(text='<b>Initial</b>',font=dict(color="white")), row=1, col=2, side='top')
fig.update_xaxes(title=dict(text='<b>Final</b>',font=dict(color="white")), row=1, col=3, side='top')

fig.update_yaxes(title=dict(text='<b>R1</b>',font=dict(color="white")), row=1, col=1)
fig.update_yaxes(title=dict(text='<b>R2</b>',font=dict(color="white")), row=2, col=1)
fig.update_yaxes(title=dict(text='<b>R3</b>',font=dict(color="white")), row=3, col=1)
fig.update_yaxes(title=dict(text='<b>R4</b>',font=dict(color="white")), row=4, col=1)


print('This (hidden) cell (re)generates Figure-5')
#plot(fig,filename='fig5.html')
iplot(fig)


This (hidden) cell (re)generates Figure-5


In [74]:
%use Python3 

import time, sys
tb = widgets.ToggleButtons(
    options=['Show Saved Figure', 'Regenerate Figure'],
    description='FIG5:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Loads pre-executed HTML image.', 'Re-generates image'],
)

@interact
def clicked(val=tb):
    if val == 'Show Saved Figure':
        clear_output(wait=True)
        display(IFrame(src='./fig5.html', width=800, height=800,scrolling='no'))     
    elif val == 'Regenerate Figure':
        display(widgets.HTML(
        value="<i>Cells responsible for regenerating the figure will be executed.</i> <br> <b>Once the run has finished, output will be rendered in the cell above.</b>",
        description='Please wait:',
        ))
        for i in reversed(range(6)):
            time.sleep(1)
            print("Execution will start in: ",i,sep='',end="\r",flush=True)
        display(Javascript("Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,IPython.notebook.get_selected_index()+1)"))



interactive(children=(ToggleButtons(button_style='info', description='FIG5:', options=('Show Saved Figure', 'R…

<div class="alert alert-info">
  <strong><span class="badge">5</span></strong> <b>Cardiac data</b>
</div>

In [52]:
# =============================================================================
# Perform SENSE on cardiac data with different subsamplings. 
# =============================================================================

heart = load('/tmp/rrsg_challenge/heart_radial_55proj_34ch.mat');
heart_raw = heart.rawdata;
heart_trajectory = heart.trajectory; 

disp('Permuting cardiac data...');
heart_raw = permute(heart_raw,[4,3,2,1]); 
heart_trajectory = permute(heart_trajectory,[3,2,1]);
[~,nFE_H,nSpokes_H,nCh_H] = size(heart_raw);
imsize_heart = 240; 

disp('Subsampling cardiac data...');
[heartRD_x2, heartTR_x2] = subSample(heart_raw,heart_trajectory,2,55);
[heartRD_x3, heartTR_x3] = subSample(heart_raw,heart_trajectory,3,55);
[heartRD_x4, heartTR_x4] = subSample(heart_raw,heart_trajectory,4,55);

disp('Calculating DCF...');
dcf_H = abs(heart_trajectory(1,:,:) + 1i*heart_trajectory(2,:,:));
dcf_H = dcf_H/max(dcf_H(:));

disp('Calculating sensitivity profiles...');
im = bart('nufft -d240:240:1 -a -t',heart_trajectory,heart_raw);
im_ks = bart('fft -u 7', im);
heart_calib = bart('caldir 20',im_ks);

disp('Calculating intensity correction matrix...');
IH = abs(bart('rss 8',heart_calib.*heart_calib));
IH = 1./sqrt(IH);

# ==> OPTION-1: Re-calculate for cardiac data. (SLOWER)
%{
disp('Performing SENSE reconstruction...');
[imh,~,~] = main_sense(heart_raw,heart_calib,heart_trajectory,IH,dcf_H,240,10,0.01);
[imh2,~,~] = main_sense(heartRD_x2,heart_calib,heartTR_x2,IH,dcf_H(:,:,1:2:end),240,10,0.01);
[imh3,~,~] = main_sense(heartRD_x3,heart_calib,heartTR_x3,IH,dcf_H(:,:,1:3:end),240,10,0.01);
[imh4,~,~] = main_sense(heartRD_x4,heart_calib,heartTR_x4,IH,dcf_H(:,:,1:4:end),240,10,0.01);
%}

# ==> OPTION-2: Re-load processing output (FASTER)
disp('Loading previous outputs...');
load('cardiac_out.mat')

In [ ]:
%use Python3
%get imh --from Octave
%get imh2 --from Octave
%get imh3 --from Octave
%get imh4 --from Octave

# =============================================================================
# To regenerate Figure-6, comment in the following code block and run the cell. 
# =============================================================================


"""
fig = subplots.make_subplots(rows=1, cols=4, print_grid=False,vertical_spacing = 0.02, horizontal_spacing=0.04)

fig.append_trace(heatmap_trace2(abs(imh), 'R1', 240, 240,'Viridis'),1,1)
fig.append_trace(heatmap_trace2(abs(imh2), 'R2', 240, 240,'Viridis'),1,2)
fig.append_trace(heatmap_trace2(abs(imh3), 'R3', 240, 240,'Viridis'),1,3)
fig.append_trace(heatmap_trace2(abs(imh4), 'R4', 240, 240,'Viridis'),1,4)

fig.update_layout(height=400, width=900, title=dict(text='<b>Figure-6</b>',font=dict(color="skyblue")),paper_bgcolor='#000000')

for ii in range(4):
    exec('fig[\'layout\'][\'xaxis' + str(ii+1) + '\'].update(showgrid = False, showline=False, zeroline = False, showticklabels = False, ticks = \'\')')
    exec('fig[\'layout\'][\'yaxis' + str(ii+1) + '\'].update(showgrid = False, zeroline = False, showline=False, showticklabels = False, ticks = \'\')')

fig.update_xaxes(title=dict(text='<b>R1</b>',font=dict(color="white")), row=1, col=1, side='top')
fig.update_xaxes(title=dict(text='<b>R2</b>',font=dict(color="white")), row=1, col=2, side='top')
fig.update_xaxes(title=dict(text='<b>R3</b>',font=dict(color="white")), row=1, col=3, side='top')
fig.update_xaxes(title=dict(text='<b>R4</b>',font=dict(color="white")), row=1, col=4, side='top')

#plot(fig,filename='fig6.html')
iplot(fig)
"""

In [76]:
%use Python3 
display(IFrame(src='./fig6.html', width=900, height=400,scrolling='no'))     